In [1]:
#Importar Bibliotecas
import numpy as np
import pandas as pd
from lxml import html
import re
import requests
from bs4 import BeautifulSoup 

In [2]:
url = 'http://lattes.cnpq.br/2503278782954156'
page = requests.get(url)
#200 quer dizer que fez o download correto da pagina
page.status_code

200

In [3]:
#Parse o HTML
soup = BeautifulSoup(page.text, 'html.parser')

In [8]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="pt-BR">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script data-dtconfig="reportUrl=/dynaTraceMonitor|bandwidth=300|agentUri=/buscatextual/ruxitagentjs_2bnr_0.js|rid=RID_-129341709|rpid=1140290984|domain=cnpq.br" src="/buscatextual/ruxitagentjs_2bnr_0.js" type="text/javascript">
  </script>
  <link href="images/v2/fav_ico_lattes.ico" rel="Shortcut Icon"/>
  <meta content="Sistema de Currículos Lattes 2.0 (CNPq)" http-equiv="generator"/>
  <meta content="currículo,curriculo,curriculum,cv,vitae,lattes,produção,producao,científica,cientifica,Brasil" http-equiv="keywords"/>
  <title>
   Currículo do Sistema de Currículos Lattes (Carlos Leite de Souza)
  </title>
  <link href="css/import.css" rel="stylesheet" type="text/css"/>
  <link href="css/tablet-mobile.css" rel="stylesheet" type="text/css"/>
  <link href="css/jquery

In [5]:
#Achar nome do titular do CV Lattes
name = soup.find(class_='nome')
name = name.text
name

'Carlos Leite de Souza'

In [9]:
#Achar todos os artigos completos publicados em periodicos
tb = soup.find_all('div', class_='artigo-completo')

In [215]:
#fazer uma lista com cada um dos anos e ISSN
ano=[]
for i in tb:
    inf_ano = i.find_all('span', class_='informacao-artigo')[2]
    ano.append(inf_ano.text)
print('Linhas Ano: ', len(ano))
issn=[]
for i in tb:
    try:
        inf_issn = i.img.get('data-issn')
    except AttributeError:
        issn.append(None)
    else:
        issn.append(inf_issn)
print('Linhas ISSN: ',len(issn))

Linhas Ano:  54
Linhas ISSN:  54


In [216]:
#fazer uma lista com cada um dos artigos completos
prod = []
for i in tb:
    prod.append(i.text)
print('Linhas Prod: ',len(prod))

Linhas Prod:  54


In [217]:
#remover duplicados de cada artigo
prod = [re.sub('\n\d{1,2}\.', '', i) for i in prod]
prod = [re.sub('\n', '', i) for i in prod]
prod = [re.sub('.*\.\d{4}', '',i) for i in prod]
prod = [re.sub('\d{1,2}\w*\,\s\w*\B\d{4}', '',i) for i in prod]
prod = [re.sub('\d{1,2}\w{2,}\s\w{2,}\d{4}', '',i) for i in prod]
prod = [re.sub('\w{2,}\s\w{2,}\,\s[\wÀ-ú]*\d{4}', '',i) for i in prod]
prod = [re.sub('\w{2,}\,\s[\wÀ-ú]*\s[\wÀ-ú]*\d{4}', '',i) for i in prod]

In [219]:
#duplicar o nome para o tamanho da lista dos artigos publicados
name_list = [name] * len(prod)
print('Linhas Nome: ', len(name_list))

Linhas Nome:  54


In [220]:
#fazer um df com listas nome e artigos cada um com seu header
prod_df = pd.DataFrame(
    {'Nome': name_list,
     'Ano': ano,
     'ArtigosCompletos': prod,
     'ISSN': issn
    })

In [221]:
prod_df.to_excel('prod.xlsx')